# Test Notebook to work on configurations without needing to train/test every time

First of all, we need to train the RNNDisaggregator using the train data. For this example, both train and test data are consumption data of the microwave of the first REDD building.

In [1]:
import warnings; warnings.filterwarnings('ignore')

from nilmtk import DataSet

train = DataSet('C:/Users/zebzi/Documents/School/Senior_Year/EE_4951W/SeniorProject/SeniorDataset/h5_files/Zeb_test.h5')
# except:
#     train = DataSet('/SeniorDataset/h5_files/Zeb_test.h5')
    
train.set_window(start="1-1-2022", end="2-1-2022")
train_elec = train.buildings[1].elec

Next, we need to define the disaggregator model.

In [2]:
from rnndisaggregator import RNNDisaggregator
rnn = RNNDisaggregator()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


Then train the model. We need to input the train data as well as their sample period. Also, we need to pass the desired number of training epochs. Finally, save the model for later use.

In [3]:
meter_key = 'kettle'
# train_mains = train_elec.mains().all_meters()[0] # The aggregated meter that provides the input
# train_meter = train_elec.submeters()['microwave'] # The microwave meter that is used as a training target

train_meter = train_elec.submeters()[meter_key]
train_mains = train_elec.mains()
print(train_elec)

rnn.train(train_mains, train_meter, epochs=1, sample_period=1)
rnn.export_model("model-Zeb_test.h5")

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='Senior Design Dataset', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='Senior Design Dataset', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=3, building=1, dataset='Senior Design Dataset', appliances=[Appliance(type='kettle', instance=1)])
  ElecMeter(instance=4, building=1, dataset='Senior Design Dataset', appliances=[Appliance(type='computer', instance=1)])
  ElecMeter(instance=5, building=1, dataset='Senior Design Dataset', appliances=[Appliance(type='microwave', instance=1)])
  ElecMeter(instance=6, building=1, dataset='Senior Design Dataset', appliances=[Appliance(type='toaster', instance=1)])
  ElecMeter(instance=7, building=1, dataset='Senior Design Dataset', appliances=[Appliance(type='hair dryer', instance=1)])
)


TypeError: float() argument must be a string or a number, not 'Timestamp'

Now that the model is trained, we can use it to disaggregate energy data. Let's test it on the rest of the data from building 1.

First we use the model to predict the microwave consumption. The results are saved automatically in a .h5 datastore.

In [ ]:
# Configure the <test> dataset and set the test_mains variable
try:
    test = DataSet('ukdale.h5')
except:
    test = DataSet('/SeniorProject/neural-disaggregator/RNN/ukdale.h5')

test.set_window(start="1-1-2014", end="2-1-2014")
print(test.buildings[1].elec)
test_elec = test.buildings[1].elec
# test_mains = test_elec.mains().all_meters()[0]
test_mains = test_elec.mains()
# print(test_mains)



In [ ]:
disag_filename = 'disag-out.h5' # The filename of the resulting datastore
from nilmtk.datastore import HDFDataStore
output = HDFDataStore(disag_filename, 'w')

# test_mains: The aggregated signal meter
# output: The output datastore
# train_meter: This is used in order to copy the metadata of the train meter into the datastore
rnn.disaggregate(test_mains, output, train_meter, sample_period=1)
output.close()
# output = HDFDataStore(disag_filename, 'r') why is this code here?


Let's plot the results and compare them to the ground truth signal.

**Note:** Calling plot this way, downsamples the signal to reduce computing time. To plot the entire signal call
```
predicted.power_series_all_data().plot()
ground_truth.power_series_all_data().plot()
```

In [ ]:
result = DataSet(disag_filename)
res_elec = result.buildings[1].elec
predicted = res_elec['kettle']
ground_truth = test_elec['kettle']

import matplotlib.pyplot as plt
predicted.plot(plot_kwargs={'color':'red', 'label':'Predicted'})
ground_truth.plot(plot_kwargs={'color':'green', 'label':'Ground Truth'})
plt.show()

Finally let's see the metric results.

In [ ]:
import metrics
rpaf = metrics.recall_precision_accuracy_f1(predicted, ground_truth)
print("============ Recall: {}".format(rpaf[0]))
print("============ Precision: {}".format(rpaf[1]))
print("============ Accuracy: {}".format(rpaf[2]))
print("============ F1 Score: {}".format(rpaf[3]))

print("============ Relative error in total energy: {}".format(metrics.relative_error_total_energy(predicted, ground_truth)))
print("============ Mean absolute error(in Watts): {}".format(metrics.mean_absolute_error(predicted, ground_truth)))